In [ ]:
"""

製作一個網站，展現Lazy loading 與 Write-Through 

Redis的經典操作
單元素
    插入
    取出
操作一個外部組件，就等兩秒，用此方式在單機環境內感受出讀取寫入先後差異

https://docs.aws.amazon.com/zh_tw/AmazonElastiCache/latest/mem-ug/Strategies.html
以下範例均有設定TTL(time to live)

"""

In [ ]:
!pip install redis

In [ ]:
# 此處以Redis 的String資料型態做示範

from flask import Flask, render_template
import redis
import time

app = Flask(__name__)

redis_client = redis.from_url('redis://elasticache:6379')

@app.route('/')
def normal():
    return render_template('hello.html')

@app.route('/writing-lazy-loading/<something_you_write>')
def writing_lazy_loading(something_you_write):
    """
    Lazy Loading
    直接寫入檔案系統，不寫入快取
    
    """
    with open('lazyloading.txt', 'a', encoding='utf-8') as f:
        f.write(something_you_write)
        
    return f"{something_you_write} have been written in 'lazy loading mode'."

@app.route('/writing-write-through/<something_you_write>')
def writing_write_through(something_you_write):
    """
    Write Through
    寫入檔案系統，並將檔案內容寫入快取(設定30秒TTL)
    
    """
    with open('writethrough.txt', 'a', encoding='utf-8') as f:
        f.write(something_you_write)
    with open('writethrough.txt', 'r', encoding='utf-8') as f:
        content = " ".join(f.readlines())
        redis_client.set('cache_for_write_through', content, ex=30)
        
    return f"{something_you_write} have been written in 'write through mode'."

        
@app.route('/reading/<mode>')
def read_page(mode):
    """
    讀取內容
    若要測試Lazy Loading 則在URL最後加上lazy
    若要測試Write Through 則在URL最後加上through
    
    """
    if mode == 'lazy':
        # 從快取讀資料，若沒讀到則從檔案系統讀取，並寫入快取(設定30秒TTL)
        # 第一次讀取都會從檔案系統讀
        r = redis_client.get('cache_for_lazy_loading')
        if r:
            return r.decode()
        with open('lazyloading.txt', 'r', encoding='utf-8') as f:
            time.sleep(2)
            content = " ".join(f.readlines())
            redis_client.set('cache_for_lazy_loading', content, ex=30)
            return content
        
    elif mode == 'through':
        # 從快取讀資料，若沒讀到則從檔案系統讀取，並寫入快取(設定30秒TTL)
        # 只要快取沒到期，均會從快取讀
        r = redis_client.get('cache_for_write_through')
        if r:
            return r.decode()
        with open('writethrough.txt', 'r', encoding='utf-8') as f:
            time.sleep(2)
            content = " ".join(f.readlines())
            redis_client.set('cache_for_write_through', content, ex=30)
            return content
        
if __name__ == '__main__':
    app.run(host='0.0.0.0')